<a href="https://colab.research.google.com/github/rtajeong/2025_data_science_class/blob/main/scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 크롤링 예제

In [ ]:
import requests
from bs4 import BeautifulSoup

def simple_crawler(url):
    response = requests.get(url)
    response.raise_for_status()  # http error 발생시 예외 발생

    soup = BeautifulSoup(response.text, 'html.parser')

    links = soup.find_all('a')   # 모든 <a> 태그(링크) 찾기

    if links:
        for link in links:
            href = link.get('href')  # 링크의 'href' 속성 값 가져오기
            if href:
                print(href)

start_url = "https://www.google.com" # 크롤링 시작 웹페이지 URL(예: 구글 홈)
simple_crawler(start_url)

https://www.google.com/imghp?hl=en&tab=wi
https://maps.google.com/maps?hl=en&tab=wl
https://play.google.com/?hl=en&tab=w8
https://www.youtube.com/?tab=w1
https://news.google.com/?tab=wn
https://mail.google.com/mail/?tab=wm
https://drive.google.com/?tab=wo
https://www.google.com/intl/en/about/products?tab=wh
http://www.google.com/history/optout?hl=en
/preferences?hl=en
https://accounts.google.com/ServiceLogin?hl=en&passive=true&continue=https://www.google.com/&ec=GAZAAQ
/advanced_search?hl=en&authuser=0
/intl/en/ads/
/services/
/intl/en/about.html
/intl/en/policies/privacy/
/intl/en/policies/terms/


#  정적 페이지에서 명언 정보 추출

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "http://quotes.toscrape.com/"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

quotes = soup.select("div .quote")  # class="quote"
print (f"There are {len(quotes)} quotes on the page.")

for num, quote in enumerate(quotes, 1):
    text = quote.select_one(".text").text
    author = quote.select_one(".author").text
    print(f"{num}. {text} - {author}")


There are 10 quotes on the page.
1. “The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.” - Albert Einstein
2. “It is our choices, Harry, that show what we truly are, far more than our abilities.” - J.K. Rowling
3. “There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.” - Albert Einstein
4. “The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.” - Jane Austen
5. “Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.” - Marilyn Monroe
6. “Try not to become a man of success. Rather become a man of value.” - Albert Einstein
7. “It is better to be hated for what you are than to be loved for what you are not.” - André Gide
8. “I have not failed. I've just found 10,000 ways that won't work.” - Thomas A. Edison
9. “A woman is like a tea bag; you never kn

# 오픈 API (기상청 단기 예보)
- 많은 공공기관 또는 민간 서비스는 오픈 API를 통해 구조화된 데이터를 제공한다. HTML을 분석하지 않아도 되므로 안정적이고 정확하게 원하는 정보를 얻을 수 있다는 장점이 있다. 특히 공공데이터 포털, 기상청, 뉴스, 금융 등의 분야에서 매우 유용하게 활용되고 있다.
- 오픈 API는 일반 사용자에게 공개된 API로, 보통 인증키 발급을 통해 사용할 수 있다.
- 대부분의 오픈 API는 JSON 또는 XML 형식으로 데이터를 제공한다.

## 공공데이터 포털(https://www.data.go.kr) 에서 인증키 발급받기
1. 공공데이터 포털 회원가입 및 로그인 후 인증키를 발급받는다.
2. 원하는 공공데이터 검색 및 활용 신청
   - 데이터 검색: 포털의 검색창을 통해 원하는 공공데이터를 검색한다. 예를 들어, "날씨", "미세먼지", "버스", "지하철" 등으로 검색
   - 검색 결과 중 Open API 탭에서 원하는 데이터셋을 찾아 클릭.
   - 선택한 API 상세 페이지에서 "활용 신청" 버튼을 클릭
3. 인증키(Service Key) 발급 및 확인
   - 활용 신청이 완료되면 마이페이지에서 "API 키 발급/관리" 또는 "인증키 발급 현황" 메뉴에서 발급된 인증키를 확인 가능
   - 인증키는 일반적으로 인코딩된(Encoded) 버전과 디코딩된(Decoded) 버전이 제공되는데, API 호출 시 (특별히 지정되어 있지 않으면) 인코딩된 키를 주로 사용한다.
4. API 명세 확인 및 테스트
   - 각 Open API 상세 페이지에는 해당 API를 호출하는 데 필요한 요청 변수(Request Parameters), 응답 형식(Response Format), 샘플 URL 등이 됨.
   - 샘플 URL 제공됨.

## params 딕셔너리를 이용한 API 호출
- 이 방식은 API 요청의 파라미터들을 딕셔너리 형태로 관리하며 requests 라이브러리가 URL을 구성하게 하는 일반적인 방법 (키의 숨겨진 공백 문제를 해결하기 위해 my_key.strip()을 적용)
- 특정 환경이나, 서비스 키 문자열에 눈에 보이지 않는 미세한 공백 또는 특수 문자가 포함되어 있을 경우, 파이썬 코드에서는 키가 유효하지 않다고 인식되어 SERVICE_KEY_IS_NOT_REGISTERED_ERROR 오류 메시지를 받을 수 있다.
- (주의) 이 사이트는 하루 정보만을 제공하기 때문에 실행 전 날짜 정보 (base_date)를 고쳐 주어야 한다.

In [ ]:
import requests
import pandas as pd

# 1. API 호출 정보 설정
# 주의: 복사-붙여넣기 시 앞뒤 공백이 포함되지 않도록 .strip()을 사용한다.
my_key = "UrcOXfOckvYo8hX%2F8AmlgQ8nIUSgSPH1u%2BuX7Z%2BkObecxxvDE8dHnkY3LQH2%2FPfOy8%2FM4LBA7dwd4eVpRTyQ4g%3D%3D".strip()
base_url = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst"

# API 요청에 필요한 파라미터들을 딕셔너리 형태로 정의한다 (해당 사이트의 샘플 코드 참고)
params = {
    'serviceKey': my_key,
    'pageNo': '1',
    'numOfRows': '10',
    'dataType': 'JSON',
    'base_date': '20250710', # 예시 날짜 (API 명세에 맞는 형식)
    'base_time': '0600',    # 예시 시간 (API 명세에 맞는 형식)
    'nx': '55', # 예시 지점의 X좌표 (기상청 격자 좌표)
    'ny': '127' # 예시 지점의 Y좌표 (기상청 격자 좌표)
}

# 2. API 요청 및 응답 처리
# requests 라이브러리가 base_url과 params 딕셔너리를 조합하여 최종 URL을 구성하고 요청한다.
response = requests.get(base_url, params=params)

# 3. HTTP 상태 코드 확인 및 데이터 파싱
if response.status_code == 200:
    # 응답이 성공이면 파싱
    json_data = response.json()
    # 필요한 데이터는 'response' > 'body' > 'items' > 'item' 경로에 있다.
    items = json_data['response']['body']['items']['item']
    # Pandas DataFrame으로 변환하여 데이터를 표 형태로 정리한다.
    df = pd.DataFrame(items)
    print("API 호출 성공 및 DataFrame 생성 완료")
    print(df)
else:
    # 응답이 실패(200이 아님)한 경우 상태 코드를 출력한다.
    print(f"API 요청 실패: HTTP 상태 코드 {response.status_code}")
    print(f"서버 응답 내용: \n{response.text}") # 실패 시 서버가 보낸 메시지 확인

API 요청 실패: HTTP 상태 코드 404
서버 응답 내용: 
<!DOCTYPE html>
<html><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
<title>ìì¤í ì ê²</title>
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no">
<meta name="author" content="êµ­ê°ì ë³´ììê´ë¦¬ì">
<!-- <link rel="shortcut icon" href="/favicon.ico"> -->
<style>
html,body { width:100%; height:100%; margin:0; padding:0; font-size:1em; line-height:1.5; color:#666; font-family: "Noto Sans", "Malgun Gothic", "ë§ìê³ ë", "ëì" }
.wrapper { width:100%; height:100%; position:relative; }


.PZwindow {
	border:1px solid #ddd;
	background-color:#fff;
    position:absolute;
	/*position:fixed !important;*/
    z-index:99999;
}
.PZwindow * { 
	font-family: "NotoKR", "Noto Sans", "Malgun Gothic", "ë§ìê³ ë", "ëì" !important;
	font-size:1em; 
	line-height:1.5; 
	color:#2e2e38;
	letter-spacing:-0.05em; 
}
.

## full_url을 이용한 API 호출
- 브라우저에서 직접 테스트하여 완벽하게 작동하는 전체 URL 문자열을 직접 사용한다.

In [ ]:
import requests
import pandas as pd

# 1. API 호출 정보 설정 (예시 날짜/시간 고정 :2025년7월10일 06:00 시)
my_key = "UrcOXfOckvYo8hX%2F8AmlgQ8nIUSgSPH1u%2BuX7Z%2BkObecxxvDE8dHnkY3LQH2%2FPfOy8%2FM4LBA7dwd4eVpRTyQ4g%3D%3D".strip()
base_url = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst"

# 모든 파라미터가 포함된 완전한 URL
full_api_url = base_url + \
    "?serviceKey=" + my_key + \
    "&pageNo=1&numOfRows=10&dataType=JSON" + \
    "&base_date=20250711&base_time=0600" + \
    "&nx=55&ny=127"

# 2. API 요청 및 응답 처리
response = requests.get(full_api_url)

# 3. 상태 코드 확인 및 DataFrame으로 변환
if response.status_code == 200:
    json_data = response.json()
    # 'response' -> 'body' -> 'items' -> 'item' 경로로 실제 데이터 접근
    items = json_data['response']['body']['items']['item']
    df = pd.DataFrame(items)
    print("API 호출 성공 및 DataFrame 생성 완료 (full_url 방식):")
    print(df)
else:
    print(f"API 요청 실패: HTTP 상태 코드 {response.status_code}")

API 요청 실패: HTTP 상태 코드 404


In [ ]:
json_data

# 동적 스크래핑

## requests 라이브러리로 시도하기: 한계점

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "http://quotes.toscrape.com/scroll"

response = requests.get(url)
if response.status_code != 200:
    print(f"Error: Failed to fetch the page. Status code: {response.status_code}")
    exit()

soup = BeautifulSoup(response.text, 'html.parser')

# Find all 'div' tags with the class 'quote'.
# In this specific dynamic page's initial HTML, these might not be present.
extracted_data = []
quotes = soup.find_all('div', class_='quote')

if quotes:
    print(f"Found {len(quotes)} quotes (requests - initial load):")
    for quote_div in quotes:
        # Find the quote text (span tag with class='text') within each quote div.
        text_element = quote_div.find('span', class_='text')
        text = text_element.get_text(strip=True) if text_element else 'N/A'

        # Find the author's name (small tag with class='author') within each quote div.
        author_element = quote_div.find('small', class_='author')
        author = author_element.get_text(strip=True) if author_element else 'N/A'

        extracted_data.append({'Quote': text, 'Author': author})
        print(f"- {text[:50]}... (by {author})") # Print example output
else:
    print("Error: Could not find quote elements (div class='quote') in the HTML.")
    print("This confirms the content is dynamically loaded by JavaScript.")

if extracted_data:
    df_requests = pd.DataFrame(extracted_data)
    print("\nQuotes data fetched by requests (DataFrame, first 5 rows):")
    print(df_requests.head())
    print(f"\nTotal quotes fetched by requests: {len(df_requests)}.")
else:
    print("No data extracted by requests.")

Error: Could not find quote elements (div class='quote') in the HTML.
This confirms the content is dynamically loaded by JavaScript.
No data extracted by requests.


- request 가 실패하는 이유:
  - 이 스크립트가 로드되는 최초 HTML(requests가 받아오는 내용)에는 <div class="quote">와 같은 명언 태그가 거의 없다. (또는 아주 초기 몇 개만 있을 수 있다. quotes.toscrape.com/scroll의 경우, 이 스크립트가 첫 10개의 명언도 API로 가져와 append 한다. 즉, 'requests'가 받은 HTML에는 명언 태그가 하나도 없다.)
  - requests는 단순히 response.text에 담긴 HTML 문자열과 JavaScript 코드 자체만 가져올 뿐이고, 브라우저처럼 이 JavaScript 코드를 실행시키지 않는다. 따라서 JavaScript가 API를 호출하고 HTML을 동적으로 생성하여 append하는 과정이 requests에게는 보이지 않는다.
  - response.text에 script 코드가 보이고 명언 태그가 보이지 않는다는 것이 정확히 이 동적 페이지의 특징이며, requests의 한계를 명확하게 보여주는 증거이다.

## 네트워크 트래픽 분석을 통한 원본 데이터(API) 스크래핑

```
Network tab analysis 를 통해 찾은 url: http://quotes.toscrape.com/api/quotes?page=1
```

In [ ]:
import requests
import pandas as pd

# The API URL discovered through network tab analysis
# base_api_url = "http://quotes.toscrape.com/api/quotes?page=1"
base_api_url = "http://quotes.toscrape.com/api/quotes"

all_quotes_data_api = [] # List to store all fetched quote data
page_num = 1             # Starting page number for the API

while True:
    print(f"Fetching API page {page_num} data...")
    params = {
        'page': page_num
    }

    response = requests.get(base_api_url, params=params)

    # Check if the HTTP status code is not 200 (OK).
    if response.status_code != 200:
        print(f"Error during API request: Status code {response.status_code}")
        break # Exit loop if API request fails

    json_data = response.json()

    # Check the API response structure: quotes list is under the 'quotes' key
    quotes_on_page = json_data.get('quotes', [])

    if not quotes_on_page: # If no more quotes, break the loop
        print(f"No more quotes found on API page {page_num}. Ending crawl.")
        break

    for quote_info in quotes_on_page:
        quote_text = quote_info.get('text')
        author_name = quote_info.get('author', {}).get('name')
        all_quotes_data_api.append({'Quote': quote_text, 'Author': author_name})

    page_num += 1 # Move to the next page

print(f"Total quotes fetched from API: {len(all_quotes_data_api)}")

if all_quotes_data_api:
    df_api = pd.DataFrame(all_quotes_data_api)
    print("\nQuotes data fetched by direct API call (DataFrame, first 20 rows):")
    print(df_api.head(20))
    print(f"\nTotal quotes fetched: {len(df_api)}.")
else:
    print("No quotes data fetched from API.")

Fetching API page 1 data...
Fetching API page 2 data...
Fetching API page 3 data...
Fetching API page 4 data...
Fetching API page 5 data...
Fetching API page 6 data...
Fetching API page 7 data...
Fetching API page 8 data...
Fetching API page 9 data...
Fetching API page 10 data...
Fetching API page 11 data...
No more quotes found on API page 11. Ending crawl.
Total quotes fetched from API: 100

Quotes data fetched by direct API call (DataFrame, first 20 rows):
                                                Quote               Author
0   “The world as we have created it is a process ...      Albert Einstein
1   “It is our choices, Harry, that show what we t...         J.K. Rowling
2   “There are only two ways to live your life. On...      Albert Einstein
3   “The person, be it gentleman or lady, who has ...          Jane Austen
4   “Imperfection is beauty, madness is genius and...       Marilyn Monroe
5   “Try not to become a man of success. Rather be...      Albert Einstein
6   “It is 

## Selenium in Colab
- (1) a simple 예제

In [ ]:
# Selenium 및 필요한 라이브러리 설치
!pip install selenium webdriver_manager beautifulsoup4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.15.0
    Uninstalling typing_extensions-4.15.0:
      Successfully uninstalled typing_extensions-4.15.0


In [ ]:
# 라이브러리 임포트
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

print("Selenium 기본 개념 예제 실행 준비 중...")

# --- 1. Chrome 옵션 설정 (Colab에서는 필수) ---
chrome_options = Options()
chrome_options.add_argument("--headless")         # GUI 없이 백그라운드 실행
chrome_options.add_argument("--no-sandbox")       # Colab 환경 필수
chrome_options.add_argument("--disable-dev-shm-usage") # 메모리 사용 최적화
chrome_options.add_argument("--disable-gpu")      # GPU 사용 비활성화
chrome_options.add_argument("--window-size=1920,1080") # 창 크기 설정
chrome_options.add_argument("--incognito")        # 시크릿 모드

# --- 2. WebDriver 초기화 (ChromeDriverManager가 WebDriver를 알아서 찾아 설치) ---
# Selenium 로봇 팔을 생성하고, 크롬 브라우저를 제어할 준비를 합니다.
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

print("WebDriver 초기화 완료. 브라우저가 실행되었습니다 (Headless 모드).")

# --- 3. 웹페이지 열기 ---
# 로봇 팔에게 특정 URL로 이동하라고 명령합니다.
url = "https://www.naver.com" # 간단한 예시로 네이버 사용
print(f"Opening webpage: {url}")
driver.get(url)

# 페이지 로드를 위해 잠시 기다립니다. (필수는 아니지만 안정성을 위해)
time.sleep(2)

# --- 4. 현재 페이지의 HTML 소스 가져오기 ---
# 로봇 팔이 현재 브라우저 화면에 보이는 HTML 전체를 복사해 옵니다.
html_content = driver.page_source
print("HTML content fetched.")

# --- 5. BeautifulSoup으로 HTML 파싱 및 데이터 추출 ---
# 복사해 온 HTML을 BeautifulSoup으로 요리하여 원하는 정보를 찾습니다.
soup = BeautifulSoup(html_content, 'html.parser')

# 웹페이지의 제목(title 태그)을 찾아봅니다.
title_tag = soup.find('title')
if title_tag:
    page_title = title_tag.get_text(strip=True)
    print(f"Page Title: {page_title}")
else:
    print("Page title not found.")

# --- 6. 브라우저 닫기 ---
# 모든 작업이 끝나면, 로봇 팔에게 브라우저를 닫으라고 명령합니다.
# 이 과정을 잊으면 백그라운드에서 브라우저가 계속 실행되어 컴퓨터 자원을 소모합니다.
driver.quit()
print("Browser closed. Selenium session ended.")

Selenium 기본 개념 예제 실행 준비 중...


WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
#0 0x5ae51be284e3 <unknown>
#1 0x5ae51bb57c76 <unknown>
#2 0x5ae51bb7e757 <unknown>
#3 0x5ae51bb7d029 <unknown>
#4 0x5ae51bbbbccc <unknown>
#5 0x5ae51bbbb47f <unknown>
#6 0x5ae51bbb2de3 <unknown>
#7 0x5ae51bb882dd <unknown>
#8 0x5ae51bb8934e <unknown>
#9 0x5ae51bde83e4 <unknown>
#10 0x5ae51bdec3d7 <unknown>
#11 0x5ae51bdf6b20 <unknown>
#12 0x5ae51bded023 <unknown>
#13 0x5ae51bdbb1aa <unknown>
#14 0x5ae51be116b8 <unknown>
#15 0x5ae51be11847 <unknown>
#16 0x5ae51be21243 <unknown>
#17 0x7e26d451aac3 <unknown>


- 요약: Selenium의 동작 방식
  - driver = webdriver.Chrome(...): "크롬 브라우저를 제어할 로봇 팔을 준비해!"
  - driver.get(url): "그 로봇 팔로 이 주소로 이동해!"
  - time.sleep(2): "잠깐 기다려줘. 웹페이지가 다 로드되고 JavaScript도 실행될 시간을 줘." (더 정교한 대기 방법도 있지만, 초보자 예제에서는 time.sleep이 직관적입니다.)
  - html_content = driver.page_source: "지금 브라우저 화면에 보이는 모든 HTML 내용을 나에게 줘!"
  - soup = BeautifulSoup(html_content, 'html.parser'): "받아온 HTML을 BeautifulSoup에게 넘겨줘서 쉽게 요리할 수 있게 해줘."
  - driver.quit(): "할 일 다 했으니 브라우저 닫고 로봇 팔 작업 끝내!"

- (2) quotes.toscrape.com/scroll 예제 (generated by Gemini AI)
  - http://quotes.toscrape.com/scroll 페이지를 스크롤하면서 각 스크롤 후 현재 페이지에 로드된 명언들 중 중복을 제외하고 수집하고, 최종적으로 수집된 명언들 중 첫 20개를 출력하는 코드이다.
  - 이 코드는 무한 스크롤을 통해 새로운 명언이 더 이상 로드되지 않을 때까지 페이지를 계속 스크롤하며 데이터를 수집한다.

In [ ]:
# Selenium 및 필요한 라이브러리 설치
!pip install selenium
!pip install webdriver_manager
!pip install beautifulsoup4 pandas

In [ ]:
# 필요한 라이브러리 임포트
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

# Selenium의 WebDriverWait 및 expected_conditions 임포트 (요소가 나타날 때까지 대기)
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By # By를 사용하여 요소를 찾습니다.

print("Google Colab 환경에서 Selenium 설정 시작...")

# --- 1. Chrome 옵션 설정 (Headless 모드) ---
# Selenium 로봇 팔이 사용할 브라우저의 특성을 정의합니다.
chrome_options = Options()
chrome_options.add_argument("--headless")           # 브라우저 창을 띄우지 않고 백그라운드에서 실행
chrome_options.add_argument("--no-sandbox")         # Colab과 같은 리눅스 환경에서 필수적인 보안 설정 해제 (안전)
chrome_options.add_argument("--disable-dev-shm-usage") # 메모리 사용을 최적화하여 오류 방지
chrome_options.add_argument("--disable-gpu")        # GPU 가속 비활성화
chrome_options.add_argument("--window-size=1920,1080") # 가상 브라우저 창의 크기 설정 (일관된 렌더링을 위해)
chrome_options.add_argument("--incognito")          # 시크릿 모드로 브라우저 시작

# --- 2. WebDriver 초기화 ---
# Selenium 로봇 팔을 생성하고, 크롬 브라우저를 제어할 준비를 합니다.
# ChromeDriverManager().install()이 현재 크롬 버전에 맞는 WebDriver를 자동으로 다운로드하고 설정합니다.
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

print("Selenium WebDriver 초기화 완료. 브라우저가 실행되었습니다 (Headless 모드).")

url = "http://quotes.toscrape.com/scroll"
all_unique_quotes_data = [] # 모든 고유한 명언 데이터를 저장할 리스트
seen_quotes_set = set()      # 중복 명언을 추적하기 위한 집합(set)

print(f"\n--- Scraping quotes from {url} using Selenium ---")

# --- 3. 웹페이지 열기 ---
# 로봇 팔에게 특정 URL로 이동하라고 명령합니다.
driver.get(url)
print(f"Accessed page: {url}")

# --- 4. 초기 페이지 로드 대기 ---
# JavaScript에 의해 명언들이 동적으로 로드되므로, 최소한 하나의 명언 요소가 나타날 때까지 기다립니다.
# 최대 15초까지 기다리며, 'quote' 클래스를 가진 요소가 DOM에 나타나면 다음 코드를 실행합니다.
WebDriverWait(driver, 15).until(
    EC.presence_of_element_located((By.CLASS_NAME, 'quote'))
)
print("Initial quotes loaded successfully. Starting infinite scroll and data collection...")

# --- 5. 무한 스크롤 및 데이터 수집 로직 ---
last_height = driver.execute_script("return document.body.scrollHeight") # 현재 페이지의 스크롤 가능한 전체 높이
scroll_count = 0

while True:
    scroll_count += 1
    print(f"Scrolling down... (Attempt {scroll_count})")

    # 페이지를 맨 아래로 스크롤하는 JavaScript 코드 실행
    # 로봇 팔이 브라우저의 스크롤바를 맨 아래로 내리는 동작을 시뮬레이션합니다.
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # 새로운 콘텐츠가 로드될 시간을 충분히 기다립니다. (네트워크 속도에 따라 조절)
    # 이 시간 동안 브라우저는 JavaScript를 실행하고 API를 호출하여 명언들을 가져와 HTML에 추가합니다.
    time.sleep(3)

    # 스크롤 후 새로운 높이 확인
    new_height = driver.execute_script("return document.body.scrollHeight")

    # --- 6. 현재 로드된 HTML에서 명언 추출 및 저장 ---
    # 로봇 팔이 현재 브라우저 화면에 보이는 (JavaScript가 모두 실행되어 최종 렌더링된) HTML 전체를 복사해 옵니다.
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')

    # 'quote' 클래스를 가진 모든 div 태그 찾기
    quotes_on_current_page = soup.find_all('div', class_='quote')

    # 현재 페이지에서 찾은 명언들을 고유한 명언 리스트에 추가
    for quote_div in quotes_on_current_page:
        text_element = quote_div.find('span', class_='text')
        author_element = quote_div.find('small', class_='author')

        text = text_element.get_text(strip=True) if text_element else 'N/A'
        author = author_element.get_text(strip=True) if author_element else 'N/A'

        quote_tuple = (text, author) # 명언 텍스트와 저자를 튜플로 묶어 set에 저장 (set 은 중복 방지)
        if quote_tuple not in seen_quotes_set: # 이전에 본 적 없는 명언이면 추가
            all_unique_quotes_data.append({'Quote': text, 'Author': author})
            seen_quotes_set.add(quote_tuple) # set에 추가하여 중복 방지

    # --- 7. 더 이상 스크롤해도 높이가 변하지 않으면 반복 중지 ---
    if new_height == last_height:
        print("No more new content loaded after scrolling. All quotes are likely fetched.")
        break
    last_height = new_height # 다음 스크롤을 위해 현재 높이를 저장

print(f"\nTotal unique quotes collected: {len(all_unique_quotes_data)}.")

# --- 8. 수집된 명언을 DataFrame으로 변환 및 원하는 형식으로 출력 ---
if all_unique_quotes_data:
    df_quotes = pd.DataFrame(all_unique_quotes_data)

    # 상위 20개의 명언만 선택합니다. (만약 20개 미만이면 모두 출력)
    quotes_to_print = df_quotes.head(20)

    print("\n--- Collected Quotes (First 20): ---")
    # DataFrame을 요청하신 형식으로 출력합니다. (인덱스 포함, 전체 열 표시)
    print(quotes_to_print)
else:
    print("No quotes were collected.")

# --- 9. WebDriver 종료 ---
# 모든 작업이 끝나면, 로봇 팔에게 브라우저를 닫으라고 명령합니다.
driver.quit()
print("Selenium WebDriver closed. Task completed.")

In [ ]:
# Install headless Chrome
!apt-get update
!apt install -y google-chrome-stable

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,006 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://cli.github.com/packages stable/main amd64 Packages [346 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,264 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,310 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InR